In [1]:
# if colab

# !pip install pybullet
# !pip install gym
# !apt-get install python-opengl -y
# !apt install xvfb -y
# !pip install gym pyvirtualdisplay > /dev/null 2>&1
# !pip install -q git+https://github.com/tensorflow/examples.git

In [2]:
import os
import glob
import cv2
import tensorflow as tf 
from tensorflow.keras import layers, models
import numpy as np 
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
import pybullet_envs
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

In [3]:
seed = 654765645
np.random.seed(seed)
tf.random.set_seed(seed)

# check if GPU
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# colab

# from google.colab import drive
# drive.mount('/content/drive')

# root_dir = "drive/My Drive/"
# base_dir = root_dir + 'CPCtesting'
# os.makedirs(base_dir,exist_ok=True)

# train_dir = base_dir + '/train'
# os.makedirs(train_dir,exist_ok=True)

# model_dir = base_dir + '/model'
# os.makedirs(model_dir,exist_ok=True)

# if local machine
base_dir = os.getcwd()

train_dir = os.path.join(base_dir , 'train')
os.makedirs(train_dir,exist_ok=True)

model_dir = os.path.join(base_dir , 'model')
os.makedirs(model_dir,exist_ok=True)

# logs_base_dir = os.path.join(base_dir , 'logs')

log_dir = os.path.join(base_dir , 'training_logs_save')
reward_dir = os.path.join(base_dir , 'training_rewards_save')

#remove old logs
fileList1 = glob.glob(os.path.join(log_dir , "events.*"))
fileList2 = glob.glob(os.path.join(reward_dir , "events.*"))

for filePath in fileList1:
    try:
        os.remove(filePath)
    except:
        print("Error while deleting file : ", filePath)
        
for filePath in fileList2:
    try:
        os.remove(filePath)
    except:
        print("Error while deleting file : ", filePath)


# tensorboard directories
# %load_ext tensorboard
os.makedirs(log_dir, exist_ok=True)
os.makedirs(reward_dir,exist_ok=True)
# %tensorboard --logdir {logs_base_dir}

In [5]:
# get data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [6]:
class CPCModel(tf.keras.Model):
    def __init__(self,code_size, predict_terms, terms=4, units=256, image_size=64, channels=3):
        super(CPCModel, self).__init__()
        self.code_size = code_size
        self.predict_terms = predict_terms
        self.terms = terms
        self.units = units
        self.image_size = image_size
        self.channels = channels

        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.lrelu1 = tf.keras.layers.LeakyReLU()
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.lrelu2 = tf.keras.layers.LeakyReLU()
        self.conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.lrelu3 = tf.keras.layers.LeakyReLU()
        self.conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn4 = tf.keras.layers.BatchNormalization()
        self.lrelu4 = tf.keras.layers.LeakyReLU()
        self.flatten = tf.keras.layers.Flatten()
        self.dense5 = tf.keras.layers.Dense(units=256, activation='linear')
        self.bn5 = tf.keras.layers.BatchNormalization()
        self.lrelu5 = tf.keras.layers.LeakyReLU()
        self.dense6 = tf.keras.layers.Dense(units=code_size, activation='linear', name='encoder_embedding')

        self.gru = tf.keras.layers.GRU(units, return_sequences=False, name='ar_context')
        self.linear = tf.keras.layers.Dense(predict_terms*code_size, activation='linear')    
   
    def encoding(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.lrelu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lrelu2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu4(x)
        x = self.flatten(x)
        x = self.dense5(x)
        x = self.bn5(x)
        x = self.lrelu5(x)
        z = self.dense6(x)
        return z
  
    def get_context(self, x):
        z = self.encoding(x)
        z = tf.reshape(z, [-1, self.terms, self.code_size])
        c = self.gru(z)
        return c
    def get_prediction(self, x):
        c = self.get_context(x)
        z_hats = self.linear(c)
        z_hat = tf.reshape(z_hats, [-1, self.predict_terms, self.code_size])
        return z_hat

    def optimizer(self):
        pass

    def loss(self,weights,biases,labels,inputs,num_samples,num_classes): 
        loss = tf.nn.nce_loss(
        weights, biases, labels, inputs, num_sampled, num_classes, num_true=1,
        sampled_values=None, remove_accidental_hits=False, name='nce_loss')
        return loss
  
    def call(self,inputs):
        x_tm, x_tp = inputs
        x_tm = tf.reshape(x_tm, [-1, self.image_size, self.image_size, self.channels])
        x_tp = tf.reshape(x_tp, [-1, self.image_size, self.image_size, self.channels])
        z_hat = self.get_prediction(x_tm)
        z_tp = self.encoding(x_tp)
        z_tp = tf.reshape(z_tp, [-1, self.predict_terms, self.code_size])
        dot_prods = tf.reduce_mean(tf.reduce_mean(z_hat*z_tp, axis=-1), axis=-1, keepdims=True)
        probs = tf.sigmoid(dot_prods)
        return probs


  # def save(self):
  #       f1 = os.path.join(folder,'target_actor')
  #       f2 = os.path.join(folder, 'target_critic')
  #       f3 = os.path.join(folder, 'actor')
  #       f4 = os.path.join(folder, 'critic')
  #       self.target_actor.save(f1)
  #       self.target_critic.save(f2)
  #       self.actor.save(f3)
  #       self.critic.save(f4)


  # def load(self):
  #   pass

In [7]:
class ReplayBuffer():
    def __init__(self,state_space,action_space,capacity,batch):
        self.capacity = capacity
        self.batch = batch
        self.elements = 0
        
        self.avaliable_batch = 0
        self.idx = 0
        self.entries = 0 
        
        self.states = np.empty((self.capacity,state_space),dtype = np.float32)
        self.next_states = np.empty((self.capacity,state_space),dtype = np.float32)
        self.actions = np.empty((self.capacity,action_space),dtype = np.float32)
        self.rewards = np.empty((self.capacity,1),dtype = np.float32)
        self.not_dones = np.empty((self.capacity, 1), dtype=np.float32)
        
    def add(self,state,next_state,action,reward,done):
        np.copyto(self.states[self.idx], state)
        np.copyto(self.actions[self.idx], action)
        np.copyto(self.rewards[self.idx], reward)
        np.copyto(self.next_states[self.idx], next_state)
        np.copyto(self.not_dones[self.idx], not done)
        #self.avaliable_batch= (self.avaliable_batch + 1) if self.avaliable_batch < self.batch else self.batch
        #self.entries = (self.entries + 1) if self.entries < self.capacity else self.capacity
        self.idx = (self.idx + 1) % self.capacity
        self.entries = np.minimum(self.entries + 1, self.capacity)
        
    def sample(self):
        num = self.entries
        if(num > self.batch):
            num = self.batch
        #print('avaliable_batch: ',self.avaliable_batch, "entries: ", self.entries,'capacity: ', self.capacity)
        idx = np.random.choice(self.entries,size = num,replace=False)
        #print('test idx: ', idx)
        
        states = tf.convert_to_tensor(self.states[idx])
        next_states = tf.convert_to_tensor(self.next_states[idx])
        actions = tf.convert_to_tensor(self.actions[idx])
        rewards = tf.convert_to_tensor(self.rewards[idx])
        not_dones = tf.convert_to_tensor(self.not_dones[idx])
        
        return states,next_states,actions,rewards,not_dones
    
    def fill_buffer(self,timesteps,state,prev_timesteps):
        print('sim test: ',env._max_episode_steps,":",timesteps)
        for step in range(timesteps):
            action = env.action_space.sample()
            next_state, reward, done, info = env.step(action)
            np.copyto(self.states[step], state)
            np.copyto(self.actions[step], action)
            np.copyto(self.rewards[step], reward)
            np.copyto(self.next_states[step], next_state)
            np.copyto(self.not_dones[step], not done)
            state = next_state
            if(done):
                print("step: ", step)
                state = env.reset()
                print('done seeding replay buffer')            
            
        

class Actor(tf.keras.Model):
    def __init__(self,state_space,action_space,critic,actor_lr = 0.001,variance = 0.2):
        super(Actor,self).__init__()
        
        #params
        self.std = np.sqrt(variance)
        self.noise_flag = 1.0
        self.action_space = action_space
        
        #optimizer
        self.opt = tf.keras.optimizers.Adam(actor_lr)
        self.critic = critic
       
        #model
        self.dense1 = tf.keras.layers.Dense(400,
                                            #input_shape = (1,1,state_space),
                                            activation = 'relu',
                                            #bias_initializer = tf.random_uniform_initializer(minval=-0.003, maxval=0.003),
                                            bias_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                     mode='fan_in', 
                                                                                                     distribution='uniform', 
                                                                                                     seed=seed
                                                                                                    ),
                                            kernel_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                       mode='fan_in', 
                                                                                                       distribution='uniform', 
                                                                                                       seed=seed)
                                           )
        self.dense2 = tf.keras.layers.Dense(300,
                                            activation='relu',
                                            #bias_initializer = tf.random_uniform_initializer(minval=-0.003, maxval=0.003),
                                            bias_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                       mode='fan_in', 
                                                                                                       distribution='uniform', 
                                                                                                       seed=seed),
                                            kernel_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                       mode='fan_in', 
                                                                                                       distribution='uniform', 
                                                                                                       seed=seed)
                                            )
        self.dense3 = tf.keras.layers.Dense(self.action_space,
                                            bias_initializer = tf.random_uniform_initializer(minval=-0.003, 
                                                                                             maxval=0.003,
                                                                                             seed = seed
                                                                                            ),
                                            kernel_initializer = tf.random_uniform_initializer(minval=-0.003, 
                                                                                               maxval=0.003,
                                                                                               seed = seed
                                                                                              )
                                           )
        
    def loss(self,states,actions):
        actions = self(states)
        #stateactions = tf.concat([states,actions],-1)
        #print("state,action shape: ",states.shape,actions.shape)
        Q = self.critic(states,actions)
        loss = - tf.reduce_mean(Q)
        return loss
    
    def update(self,states,actions):
        with tf.GradientTape() as tape:
            loss = self.loss(states,actions)

        grad = tape.gradient(loss,self.trainable_variables)
        self.opt.apply_gradients(zip(grad, self.trainable_variables))
        #print('actor loss: ', loss ,"\n" )
        return loss
    
    def set_noise_flag(self,num):
        self.noise_flag = np.float32(not not num)
    
    def continous_noise(self):
        #num = np.random.normal(0,self.std)
        #result = np.full((self.action_space,),num)
        result = np.random.normal(0,self.std,size=(self.action_space,))
        return self.noise_flag * np.clip(result,a_min = -1.0, a_max = 1.0)
    
    def call(self,x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

class Critic(tf.keras.Model):
    def __init__(self,combined_space,critic_lr = 0.001):
        super(Critic,self).__init__()
        
        # optimizer
        self.opt = tf.keras.optimizers.Adam(critic_lr)
        
        # loss
        #self.loss = tf.keras.losses.MSE
        
        
        # layers
        self.dense1 = tf.keras.layers.Dense(400,
                                            #input_shape=(1,1,combined_space),
                                            activation = 'relu',
                                            #bias_initializer = tf.random_uniform_initializer(minval=-0.003, maxval=0.003),
                                            bias_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                     mode='fan_in', 
                                                                                                     distribution='uniform', 
                                                                                                     seed=seed
                                                                                                    ),
                                            kernel_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                       mode='fan_in', 
                                                                                                       distribution='uniform', 
                                                                                                       seed=seed),
                                            kernel_regularizer=tf.keras.regularizers.l2(0.01)
                                           )

        self.dense2 = tf.keras.layers.Dense(300,
                                            activation='relu',
                                            #bias_initializer = tf.random_uniform_initializer(minval=-0.003, maxval=0.003),
                                            bias_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                     mode='fan_in', 
                                                                                                     distribution='uniform', 
                                                                                                     seed=seed
                                                                                                    ),
                                            kernel_initializer = tf.keras.initializers.VarianceScaling(scale=1.0, 
                                                                                                       mode='fan_in', 
                                                                                                       distribution='uniform', 
                                                                                                       seed=seed
                                                                                                      ),
                                            kernel_regularizer=tf.keras.regularizers.l2(0.01)
                                            )
        self.dense3 = tf.keras.layers.Dense(1,
                                            bias_initializer = tf.random_uniform_initializer(minval=-0.003, maxval=0.003),
                                            kernel_initializer = tf.random_uniform_initializer(minval=-0.003, 
                                                                                               maxval=0.003,
                                                                                               seed = seed
                                                                                              ),
                                            kernel_regularizer=tf.keras.regularizers.l2(0.01)
                                            ) 
        
    #loss
    def loss(self,actual,pred):
        result = tf.keras.losses.MSE(actual,pred)
        #print('result: ', result)
        #print('actual: ', actual.shape) # shape (16,1)
        #print('pred: ',pred.shape) # shape (16,1,1,1)
        return result
    
    def update(self,states_i,actions_i,Q_h):
        match = Q_h.shape[0]
        with tf.GradientTape() as tape:
            Q = self.call(states_i,actions_i)
            Q = tf.reshape(Q,(1,1,1,match))
            Q_h = tf.reshape(Q_h,(1,1,1,match))
            loss = self.loss(Q,Q_h)

        grad = tape.gradient(loss,self.trainable_variables)
        #grad_magnitude = tf.reduce_sum(grad)
        self.opt.apply_gradients(zip(grad, self.trainable_variables))
        #print('critic loss: ', loss ,"\n" )
        #print("check exploding gradient: ", grad)
        return loss
    
    #predict
    def call(self,states,actions):
        #x = tf.concat([states,actions],-1)
        x = self.dense1(states)
        x = tf.concat([x,actions],-1)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
    
    
class SAC(tf.keras.Model):
    def __init__(self,
                 state_space,
                 action_space,
                 capacity = 1000,
                 batch = 1, 
                 tau=0.999,
                 gamma=0.99,
                 actor_lr = 0.001, 
                 critic_lr = 0.0001,
                 variance = 1.0):
        super(SAC,self).__init__()
        # tensorboard callbacks
        self.cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4),
                   tf.keras.callbacks.ModelCheckpoint('weights/weights.{epoch:02d}-{val_binary_accuracy:.2f}.cpkt',
                                          monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True),
                   tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3),
                   tf.keras.callbacks.TensorBoard()]
        
        
        #hyperparameters
        self.batch = batch
        self.tau = tau
        self.gamma = gamma
        self.actor_lr = actor_lr
        self.critic_lr = critic_lr
        self.noise_flag = 1
        self.std = np.sqrt(variance)
        
        
        #spaces
        self.action_space = action_space
        self.state_space = state_space
        self.combined_space = self.action_space + self.state_space
        
        # replay buffer
        self.replay_buffer = ReplayBuffer(self.state_space,self.action_space,capacity,self.batch)
        
        # models
        self.critic = Critic(combined_space = self.combined_space,
                             critic_lr = critic_lr
                            )        
        self.actor = Actor(self.state_space,
                           self.action_space,
                           actor_lr=actor_lr,
                           critic = self.critic
                          )
        #self.critic.compile(optimizer = self.critic.opt,loss = self.critic.loss)
        #self.actor.compile(optimizer = self.actor.opt,loss = self.actor.loss)
        
        # target models
        self.target_actor = Actor(self.state_space,
                                  self.action_space,
                                  actor_lr=actor_lr,
                                  critic = self.critic
                                 )
        self.target_critic = Critic(combined_space = self.combined_space,
                                    critic_lr = critic_lr
                                   )         
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())
        
        #cpc
        #self.cpc = CPC(code_size=128, predict_terms=4, terms=4, units=256, image_size=64, channels=3)
    
    def store_replay(self,state,next_state,action,reward,done):
        self.replay_buffer.add(state,next_state,action,reward,done)
    
    def set_labels(self,states_i,next_states_i,actions_i,rewards_i,terminal_i):
        mu = self.target_actor(next_states_i)
        #print('ends: ', terminal)
        #print(mu,states)
#         stateactions = tf.concat([states,mu],1)
        Q_h = self.target_critic(next_states_i,mu)
        y = rewards_i + terminal_i*self.gamma * Q_h
        #y = np.concatenate(self.y,0).astype('float32') #.reshape((self.minibatch_size,1,1,1))
        #print('y: ',self.y)
        #y = tf.reshape(y,(self.replay_buffer.batch,1,1,1))
        return y 
    
        
    def discrete_random_noise(self):
        pass
    
    def update_target_weights(self):   
        tgt_critic_weight = self.target_critic.get_weights()
        tgt_actor_weight = self.target_actor.get_weights()
        actor_weight = self.actor.get_weights()
        critic_weight = self.target_actor.get_weights()
        
        for idx,(part_tgt,part_net) in enumerate(zip(tgt_critic_weight,critic_weight)):
            tgt_critic_weight[idx] = self.tau*part_tgt + (1.0-self.tau)*part_net
        
        for idx,(part_tgt,part_net) in enumerate(zip(tgt_actor_weight,actor_weight)):
            tgt_actor_weight[idx] = self.tau*part_tgt + (1.0-self.tau)*part_net
            
        self.target_actor.set_weights(tgt_actor_weight)
        self.target_critic.set_weights(tgt_critic_weight)
            
    def save(self,filename):
        self.actor.save_weights(filename)
        self.critic.save_weights(filename)
        self.target_actor.save_weights(filename)
        self.target_critic.save_weights(filename)
    
    def load(self,filename):
        self.actor.load_weights(filename)
        self.critic.load_weights(filename)
        self.target_actor.load_weights(filename)
        self.target_critic.load_weights(filename)
            

In [8]:
class DataHandler:
    def __init__(self, batch_size, terms, predict_terms=1, image_size=64, color=False, rescale=True, aug=True, is_training=True, method='cpc'):
        self.batch_size = batch_size
        self.terms = terms
        self.predict_terms = predict_terms
        self.image_size = image_size
        self.color = color
        self.rescale = rescale
        self.aug = aug
        self.is_training = is_training
        self.method = method
        self.lena = cv2.imread(os.path.join(base_dir,'lena.jpg'))
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        if self.is_training:
            self.x = x_train/255.0
            self.y = y_train
        else:
            self.x = x_test/255.0
            self.y = y_test
        self.idxs = []
        for i in range(10):
            y = y_train if self.is_training else y_test
            self.idxs.append(np.where(y == i)[0])
        self.n_samples = len(self.y)//terms if self.method == 'cpc' else len(self.y)
        self.shape = self.x.shape
        self.n_batches = self.n_samples//batch_size

    def __iter__(self):
        return self

    def __next__(self):
        return self.cpc_batch() if self.method == 'cpc' else self.benchmark_batch()

    def __len__(self):
        return self.n_batches

    def cpc_batch(self):
        img_labels = np.zeros((self.batch_size, self.terms + self.predict_terms))
        sentence_labels = np.ones((self.batch_size, 1)).astype('int32')
        for bi in range(self.batch_size):
            seed = np.random.randint(10)
            sentence = np.arange(seed, seed + self.terms + self.predict_terms) % 10
            if bi < self.batch_size//2:
                num = np.arange(10)
                predicted = sentence[-self.predict_terms:]
                for i, p in enumerate(predicted):
                    predicted[i] = np.random.choice(num[num != p], 1)
                sentence[-self.predict_terms:] = predicted % 10
                sentence_labels[bi, :] = 0
            img_labels[bi, :] = sentence
        images = self.get_samples(img_labels).reshape((self.batch_size, self.terms+self.predict_terms, self.image_size, self.image_size, 3))
        x_images = images[:, :-self.predict_terms, ...]
        y_images = images[:, -self.predict_terms:, ...]
        idx = np.random.choice(self.batch_size, self.batch_size, replace=False)
        return [x_images[idx], y_images[idx]], sentence_labels[idx]

    def get_samples(self, img_labels):
        idx = []
        for label in img_labels.flatten():
            idx.append(np.random.choice(self.idxs[int(label)], 1)[0])
        img_batch = self.x[idx, :, :]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        return img_batch

    def _aug_batch(self, img_batch):
        if self.image_size != 28:
            resized = []
            for i in range(img_batch.shape[0]):
                resized.append(cv2.resize(img_batch[i], (self.image_size, self.image_size)))
            img_batch = np.stack(resized)
        img_batch = img_batch.reshape((img_batch.shape[0], 1, self.image_size, self.image_size))
        img_batch = np.concatenate([img_batch, img_batch, img_batch], axis=1)

        if self.color:
            img_batch[img_batch >= 0.5] = 1
            img_batch[img_batch < 0.5] = 0
            for i in range(img_batch.shape[0]):
                x_c = np.random.randint(0, self.lena.shape[0] - self.image_size)
                y_c = np.random.randint(0, self.lena.shape[1] - self.image_size)
                img = self.lena[x_c:x_c+self.image_size, y_c:y_c+self.image_size]
                img = np.array(img).transpose((2, 0, 1))/255.0
                for j in range(3):
                    img[j, :, :] = (img[j, :, :] + np.random.uniform(0, 1))/2.0
                img[img_batch[i, :, :, :] == 1] = 1 - img[img_batch[i, :, :, :] == 1]
                img_batch[i, :, :, :] = img

        if self.rescale:
            img_batch = img_batch * 2 - 1
        img_batch = img_batch.transpose((0, 2, 3, 1))
        return img_batch

    def benchmark_batch(self):
        idx = np.random.choice(len(self.x), self.batch_size, replace=False)
        img_batch = self.x[idx]
        label_batch = self.y[idx]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        label_batch = label_batch.reshape((-1, 1))
        return img_batch, label_batch

In [9]:
# #train loop
# dh_train = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=True, method='cpc')
# dh_test = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=False, method='cpc')
# accuracy_metric_train = tf.keras.metrics.BinaryAccuracy()
# loss_metric_train = tf.keras.metrics.BinaryCrossentropy()
# accuracy_metric_test = tf.keras.metrics.BinaryAccuracy()
# loss_metric_test = tf.keras.metrics.BinaryCrossentropy()
# cpc = CPCModel(code_size=128, predict_terms=4, terms=4, units=256, image_size=64, channels=3)
# optim = tf.keras.optimizers.Adam(1e-3)
# cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4),
#       tf.keras.callbacks.ModelCheckpoint('weights/weights.{epoch:02d}-{val_binary_accuracy:.2f}.cpkt',
#                                           monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True),
#       tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3),
#       tf.keras.callbacks.TensorBoard()]
# cpc.compile(optimizer=optim, loss='binary_crossentropy', metrics=['binary_accuracy'])
# cpc.fit(x=dh_train, epochs=10, validation_data=dh_test, steps_per_epoch=60000//64, validation_steps=10000//64, callbacks=cb)


In [10]:
#%tensorboard --logdir {logs_base_dir}

In [11]:
# train loop params


episodes = 200
episode_steps = 1000
buffer_size = 100000
batch_size = 16

# pybullet setup
env = gym.make('HalfCheetahBulletEnv-v0')
env.render(mode = 'human')
env._max_episode_steps = episode_steps



C:\Users\dtara\anaconda3\envs\csci-7000-rl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [12]:
writer = tf.summary.create_file_writer(log_dir)
writer_reward = tf.summary.create_file_writer(reward_dir)

#get spaces
state_space = env.observation_space.shape[0]
action_space = env.action_space.shape[0]
print(state_space,action_space)

26 6


In [ ]:
state = env.reset()
sac = SAC(action_space=action_space,
          state_space=state_space,
          capacity = buffer_size,
          batch = batch_size,
          tau = 0.999,
          gamma = 0.99,
          actor_lr = 0.0001,
          critic_lr = 0.001,
          variance = 0.2)

#fill replay buffer
#env._max_episode_steps = buffer_size
#sac.replay_buffer.fill_buffer(buffer_size, state, episode_steps) # self,timesteps,state,prev_timesteps
#env._max_episode_steps = episode_steps


env = gym.wrappers.Monitor(env, "baseline_training", video_callable=lambda episode: True, force="true")
state = env.reset()

for episode in range(episodes):
    sumreward = 0
    for step in range(episode_steps):
        #print(observation)
        print('t: ',step, ' :episode: ',episode)
        #print('state: ',state)
        
        # get action
        state = tf.reshape(state,(1,1,state_space)) #,dtype='float32')
        #print(state)
        tensor_noisy_action = sac.actor(state)+sac.actor.continous_noise()
        #tensor_action = tf.clip_by_value(tensor_action, clip_value_min=-1.0, clip_value_max=1.0)

        noisy_action = tensor_noisy_action[0][0]
        #print('action: ',action)
        
        #get loss
        #q_loss = sac.critic(state,tensor_action)
        
        
        # execute action
        next_state, reward, done, info = env.step(noisy_action)
        sumreward += reward

        # store transitions
        sac.store_replay(state,next_state,noisy_action,reward,done)
        
        #print('state: ',state)
        #print('next_state: ',next_state)
        #print('action: ',action)
        #print('reward: ',reward)

        #sample minibatch from data
        states_i,next_states_i,actions_i,rewards_i,terminal_i = sac.replay_buffer.sample()
        
        #set labels y_i
        y = sac.set_labels(states_i,next_states_i,actions_i,rewards_i,terminal_i)
        #print('y: ',y)
        
        # update critic net
        q_loss = sac.critic.update(states_i, actions_i, y)

        print('q_loss: ', q_loss.numpy())
        with writer.as_default():
            tf.summary.scalar('Squared QLosses (qtarget - qval)^2', q_loss[0][0][0].numpy(),
                              step=episode * episode_steps + step + 1)
        
        #losses[episode*timesteps + t] = loss
        #losses[i_episode*timesteps+] = history.history
        
        #update actor net
        sac.actor.update(states_i, sac.actor(states_i)) #actions)
        #print('weight check: ',rl.actor.get_weights(),'\n')
        
        #update target nets
        sac.update_target_weights()
        
        state = next_state
        if done:
            state = env.reset()
            #rewards[episode] = sumreward
            #sac.save(base_dir+'/baseline_model')
            print("Episode {} finished after {} timesteps with average reward {}".format(episode,step+1,sumreward))
            with writer_reward.as_default():
                tf.summary.scalar('Episode sum reward', sumreward,step=episode)
            break
print('done') 
sac.save(base_dir+'/baseline_model')

t:  0  :episode:  0
q_loss:  [[[0.51214576]]]
t:  1  :episode:  0
q_loss:  [[[0.32530105]]]
t:  2  :episode:  0
q_loss:  [[[0.19346547]]]
t:  3  :episode:  0
q_loss:  [[[0.2349267]]]
t:  4  :episode:  0
q_loss:  [[[0.15801728]]]
t:  5  :episode:  0
q_loss:  [[[0.09860212]]]
t:  6  :episode:  0
q_loss:  [[[0.1298646]]]
t:  7  :episode:  0
q_loss:  [[[0.15899816]]]
t:  8  :episode:  0
q_loss:  [[[0.18582213]]]
t:  9  :episode:  0
q_loss:  [[[0.17287856]]]
t:  10  :episode:  0
q_loss:  [[[0.14272122]]]
t:  11  :episode:  0
q_loss:  [[[0.11208106]]]
t:  12  :episode:  0
q_loss:  [[[0.0888068]]]
t:  13  :episode:  0
q_loss:  [[[0.07490087]]]
t:  14  :episode:  0
q_loss:  [[[0.06721821]]]
t:  15  :episode:  0
q_loss:  [[[0.47394645]]]
t:  16  :episode:  0
q_loss:  [[[0.4625653]]]
t:  17  :episode:  0
q_loss:  [[[0.8621845]]]
t:  18  :episode:  0
q_loss:  [[[0.96047574]]]
t:  19  :episode:  0
q_loss:  [[[1.218885]]]
t:  20  :episode:  0
q_loss:  [[[1.167798]]]
t:  21  :episode:  0
q_loss:  [[

q_loss:  [[[0.1136747]]]
t:  177  :episode:  0
q_loss:  [[[0.06861548]]]
t:  178  :episode:  0
q_loss:  [[[0.49416927]]]
t:  179  :episode:  0
q_loss:  [[[0.1210062]]]
t:  180  :episode:  0
q_loss:  [[[0.04252842]]]
t:  181  :episode:  0
q_loss:  [[[0.04372288]]]
t:  182  :episode:  0
q_loss:  [[[0.14988744]]]
t:  183  :episode:  0
q_loss:  [[[0.18525183]]]
t:  184  :episode:  0
q_loss:  [[[0.301201]]]
t:  185  :episode:  0
q_loss:  [[[0.2611261]]]
t:  186  :episode:  0
q_loss:  [[[0.13142455]]]
t:  187  :episode:  0
q_loss:  [[[0.05618998]]]
t:  188  :episode:  0
q_loss:  [[[0.13297567]]]
t:  189  :episode:  0
q_loss:  [[[0.30005333]]]
t:  190  :episode:  0
q_loss:  [[[0.4395577]]]
t:  191  :episode:  0
q_loss:  [[[0.3201246]]]
t:  192  :episode:  0
q_loss:  [[[0.23862687]]]
t:  193  :episode:  0
q_loss:  [[[0.17535277]]]
t:  194  :episode:  0
q_loss:  [[[0.08263938]]]
t:  195  :episode:  0
q_loss:  [[[0.06569918]]]
t:  196  :episode:  0
q_loss:  [[[0.29899076]]]
t:  197  :episode:  0

q_loss:  [[[0.18571508]]]
t:  351  :episode:  0
q_loss:  [[[0.08135073]]]
t:  352  :episode:  0
q_loss:  [[[0.24254662]]]
t:  353  :episode:  0
q_loss:  [[[0.12684877]]]
t:  354  :episode:  0
q_loss:  [[[0.39597043]]]
t:  355  :episode:  0
q_loss:  [[[0.45330146]]]
t:  356  :episode:  0
q_loss:  [[[0.24661848]]]
t:  357  :episode:  0
q_loss:  [[[0.07059251]]]
t:  358  :episode:  0
q_loss:  [[[0.05086918]]]
t:  359  :episode:  0
q_loss:  [[[0.3070537]]]
t:  360  :episode:  0
q_loss:  [[[0.0923961]]]
t:  361  :episode:  0
q_loss:  [[[0.17109261]]]
t:  362  :episode:  0
q_loss:  [[[0.19715388]]]
t:  363  :episode:  0
q_loss:  [[[0.31088287]]]
t:  364  :episode:  0
q_loss:  [[[0.21144226]]]
t:  365  :episode:  0
q_loss:  [[[0.25664216]]]
t:  366  :episode:  0
q_loss:  [[[0.24182035]]]
t:  367  :episode:  0
q_loss:  [[[0.17585957]]]
t:  368  :episode:  0
q_loss:  [[[0.29477245]]]
t:  369  :episode:  0
q_loss:  [[[0.08915579]]]
t:  370  :episode:  0
q_loss:  [[[0.04178256]]]
t:  371  :episod

q_loss:  [[[0.11917534]]]
t:  524  :episode:  0
q_loss:  [[[0.08146324]]]
t:  525  :episode:  0
q_loss:  [[[0.05423318]]]
t:  526  :episode:  0
q_loss:  [[[0.24237628]]]
t:  527  :episode:  0
q_loss:  [[[0.09645288]]]
t:  528  :episode:  0
q_loss:  [[[0.28715998]]]
t:  529  :episode:  0
q_loss:  [[[0.06887515]]]
t:  530  :episode:  0
q_loss:  [[[0.17986822]]]
t:  531  :episode:  0
q_loss:  [[[0.098179]]]
t:  532  :episode:  0
q_loss:  [[[0.07066942]]]
t:  533  :episode:  0
q_loss:  [[[0.14780523]]]
t:  534  :episode:  0
q_loss:  [[[0.15473802]]]
t:  535  :episode:  0
q_loss:  [[[0.03879286]]]
t:  536  :episode:  0
q_loss:  [[[0.09935959]]]
t:  537  :episode:  0
q_loss:  [[[0.24300298]]]
t:  538  :episode:  0
q_loss:  [[[0.040072]]]
t:  539  :episode:  0
q_loss:  [[[0.30939078]]]
t:  540  :episode:  0
q_loss:  [[[0.05741322]]]
t:  541  :episode:  0
q_loss:  [[[0.2449966]]]
t:  542  :episode:  0
q_loss:  [[[0.04931071]]]
t:  543  :episode:  0
q_loss:  [[[0.06360967]]]
t:  544  :episode: 

t:  697  :episode:  0
q_loss:  [[[0.12697116]]]
t:  698  :episode:  0
q_loss:  [[[0.36982906]]]
t:  699  :episode:  0
q_loss:  [[[0.06739365]]]
t:  700  :episode:  0
q_loss:  [[[0.17727895]]]
t:  701  :episode:  0
q_loss:  [[[0.04546996]]]
t:  702  :episode:  0
q_loss:  [[[0.33565098]]]
t:  703  :episode:  0
q_loss:  [[[0.14910677]]]
t:  704  :episode:  0
q_loss:  [[[0.2802129]]]
t:  705  :episode:  0
q_loss:  [[[0.11113691]]]
t:  706  :episode:  0
q_loss:  [[[0.29497862]]]
t:  707  :episode:  0
q_loss:  [[[0.20365383]]]
t:  708  :episode:  0
q_loss:  [[[0.4209993]]]
t:  709  :episode:  0
q_loss:  [[[0.20478722]]]
t:  710  :episode:  0
q_loss:  [[[0.08837675]]]
t:  711  :episode:  0
q_loss:  [[[0.179288]]]
t:  712  :episode:  0
q_loss:  [[[0.4719761]]]
t:  713  :episode:  0
q_loss:  [[[0.03572735]]]
t:  714  :episode:  0
q_loss:  [[[0.11410885]]]
t:  715  :episode:  0
q_loss:  [[[0.14111272]]]
t:  716  :episode:  0
q_loss:  [[[0.18828443]]]
t:  717  :episode:  0
q_loss:  [[[0.2917118]]

...

t:  82  :episode:  11
q_loss:  [[[35.711365]]]
t:  83  :episode:  11
q_loss:  [[[161.0322]]]
t:  84  :episode:  11
q_loss:  [[[78.604546]]]
t:  85  :episode:  11
q_loss:  [[[16.648602]]]
t:  86  :episode:  11
q_loss:  [[[40.517654]]]
t:  87  :episode:  11
q_loss:  [[[53.050323]]]
t:  88  :episode:  11
q_loss:  [[[70.79172]]]
t:  89  :episode:  11
q_loss:  [[[22.095795]]]
t:  90  :episode:  11
q_loss:  [[[21.893808]]]
t:  91  :episode:  11
q_loss:  [[[103.89809]]]
t:  92  :episode:  11
q_loss:  [[[21.46759]]]
t:  93  :episode:  11
q_loss:  [[[7.4115286]]]
t:  94  :episode:  11
q_loss:  [[[42.57178]]]
t:  95  :episode:  11
q_loss:  [[[9.70646]]]
t:  96  :episode:  11
q_loss:  [[[40.834145]]]
t:  97  :episode:  11
q_loss:  [[[7.970331]]]
t:  98  :episode:  11
q_loss:  [[[271.92035]]]
t:  99  :episode:  11
q_loss:  [[[18.664896]]]
t:  100  :episode:  11
q_loss:  [[[56.79509]]]
t:  101  :episode:  11
q_loss:  [[[34.16626]]]
t:  102  :episode:  11
q_loss:  [[[115.87218]]]
t:  103  :episode: 

t:  256  :episode:  11
q_loss:  [[[38.31173]]]
t:  257  :episode:  11
q_loss:  [[[31.135464]]]
t:  258  :episode:  11
q_loss:  [[[43.612373]]]
t:  259  :episode:  11
q_loss:  [[[65.608696]]]
t:  260  :episode:  11
q_loss:  [[[45.52804]]]
t:  261  :episode:  11
q_loss:  [[[55.545815]]]
t:  262  :episode:  11
q_loss:  [[[115.541695]]]
t:  263  :episode:  11
q_loss:  [[[61.097263]]]
t:  264  :episode:  11
q_loss:  [[[33.145603]]]
t:  265  :episode:  11
q_loss:  [[[21.834253]]]
t:  266  :episode:  11
q_loss:  [[[12.631475]]]
t:  267  :episode:  11
q_loss:  [[[22.441599]]]
t:  268  :episode:  11
q_loss:  [[[73.01595]]]
t:  269  :episode:  11
q_loss:  [[[77.780426]]]
t:  270  :episode:  11
q_loss:  [[[32.544067]]]
t:  271  :episode:  11
q_loss:  [[[11.277407]]]
t:  272  :episode:  11
q_loss:  [[[27.64282]]]
t:  273  :episode:  11
q_loss:  [[[28.121758]]]
t:  274  :episode:  11
q_loss:  [[[33.57441]]]
t:  275  :episode:  11
q_loss:  [[[10.092092]]]
t:  276  :episode:  11
q_loss:  [[[28.75457]

q_loss:  [[[13.1827345]]]
t:  430  :episode:  11
q_loss:  [[[16.585266]]]
t:  431  :episode:  11
q_loss:  [[[27.377842]]]
t:  432  :episode:  11
q_loss:  [[[39.716053]]]
t:  433  :episode:  11
q_loss:  [[[25.262989]]]
t:  434  :episode:  11
q_loss:  [[[109.48511]]]
t:  435  :episode:  11
q_loss:  [[[32.72603]]]
t:  436  :episode:  11
q_loss:  [[[36.016888]]]
t:  437  :episode:  11
q_loss:  [[[48.352707]]]
t:  438  :episode:  11
q_loss:  [[[8.024181]]]
t:  439  :episode:  11
q_loss:  [[[85.12978]]]
t:  440  :episode:  11
q_loss:  [[[18.596725]]]
t:  441  :episode:  11
q_loss:  [[[13.056522]]]
t:  442  :episode:  11
q_loss:  [[[30.300941]]]
t:  443  :episode:  11
q_loss:  [[[34.270123]]]
t:  444  :episode:  11
q_loss:  [[[21.864134]]]
t:  445  :episode:  11
q_loss:  [[[23.830145]]]
t:  446  :episode:  11
q_loss:  [[[54.00074]]]
t:  447  :episode:  11
q_loss:  [[[28.251453]]]
t:  448  :episode:  11
q_loss:  [[[63.487793]]]
t:  449  :episode:  11
q_loss:  [[[356.39163]]]
t:  450  :episode:

q_loss:  [[[52.166756]]]
t:  602  :episode:  11
q_loss:  [[[42.646976]]]
t:  603  :episode:  11
q_loss:  [[[18.523243]]]
t:  604  :episode:  11
q_loss:  [[[101.99388]]]
t:  605  :episode:  11
q_loss:  [[[12.865169]]]
t:  606  :episode:  11
q_loss:  [[[40.467865]]]
t:  607  :episode:  11
q_loss:  [[[123.55504]]]
t:  608  :episode:  11
q_loss:  [[[7.288364]]]
t:  609  :episode:  11
q_loss:  [[[60.237164]]]
t:  610  :episode:  11
q_loss:  [[[65.0359]]]
t:  611  :episode:  11
q_loss:  [[[30.07265]]]
t:  612  :episode:  11
q_loss:  [[[29.30484]]]
t:  613  :episode:  11
q_loss:  [[[41.32432]]]
t:  614  :episode:  11
q_loss:  [[[119.443596]]]
t:  615  :episode:  11
q_loss:  [[[104.91341]]]
t:  616  :episode:  11
q_loss:  [[[65.25998]]]
t:  617  :episode:  11
q_loss:  [[[68.185555]]]
t:  618  :episode:  11
q_loss:  [[[64.78271]]]
t:  619  :episode:  11
q_loss:  [[[29.759535]]]
t:  620  :episode:  11
q_loss:  [[[37.60147]]]
t:  621  :episode:  11
q_loss:  [[[32.687065]]]
t:  622  :episode:  11


q_loss:  [[[20.997238]]]
t:  774  :episode:  11
q_loss:  [[[40.0428]]]
t:  775  :episode:  11
q_loss:  [[[52.84193]]]
t:  776  :episode:  11
q_loss:  [[[293.32507]]]
t:  777  :episode:  11
q_loss:  [[[141.75357]]]
t:  778  :episode:  11
q_loss:  [[[679.4546]]]
t:  779  :episode:  11
q_loss:  [[[117.877884]]]
t:  780  :episode:  11
q_loss:  [[[72.23043]]]
t:  781  :episode:  11
q_loss:  [[[47.81852]]]
t:  782  :episode:  11
q_loss:  [[[1560.021]]]
t:  783  :episode:  11
q_loss:  [[[345.2396]]]
t:  784  :episode:  11
q_loss:  [[[119.09002]]]
t:  785  :episode:  11
q_loss:  [[[21.582233]]]
t:  786  :episode:  11
q_loss:  [[[112.57035]]]
t:  787  :episode:  11
q_loss:  [[[533.8506]]]
t:  788  :episode:  11
q_loss:  [[[39.156715]]]
t:  789  :episode:  11
q_loss:  [[[5602.7188]]]
t:  790  :episode:  11
q_loss:  [[[1226.122]]]
t:  791  :episode:  11
q_loss:  [[[265.97452]]]
t:  792  :episode:  11
q_loss:  [[[282.34497]]]
t:  793  :episode:  11
q_loss:  [[[260.27676]]]
t:  794  :episode:  11
q

 https://datascience.stackexchange.com/questions/13216/intuitive-explanation-of-noise-contrastive-estimation-nce-loss(InfoNCE Loss )
<br>
Representation Learning with Contrastive Predictive Coding
<br>
https://github.com/gdao-research/cpc/blob/master/cpc/data_handler.py (CPC)
<br>
https://github.com/davidtellez/contrastive-predictive-coding/blob/master/train_model.py (CPC)
<br>
https://github.com/MishaLaskin/curl/blob/23b0880708c29b078b0a25e62ff31fb587587b18/utils.py#L123 (replay buffer and SAC)
<br>
https://github.com/marload/DeepRL-TensorFlow2/blob/master/A2C/A2C_Discrete.py (A2C)
<br>
https://github.com/germain-hug/Deep-RL-Keras/blob/master/A3C/a3c.py (A3C)
<br>
https://github.com/tensorflow/agents/blob/v0.5.0/tf_agents/agents/sac/sac_agent.py (SAC)
<br>
https://github.com/cookbenjamin/DDPG/blob/master/networks/critic.py (transfer the action state merge to second layer)
<br>
https://github.com/georgesung/TD3 (check expected results)
<br>
https://github.com/georgesung/TD3/blob/master/DDPG.py (param mistake)